## Evaluate.ipynb
#### **This notebook evaluates the accuracy of just one model on the IEMOCAP dataset ground truths.**

- **Transcription Model (S2T):** whisper-tiny.en
- Source: [HuggingFace](https://huggingface.co/openai/whisper-tiny)

## Setup

In [1]:
# Run
SEED = 22
FOLD = 0

# Models
TRANSCRIPTION_MODEL_NAME = "openai/whisper-tiny.en"

# Flag to enable parsing of arguments when converted to script. Set true after converting
PYTHON_SCRIPT = False

### For Conversion to .py file

In [2]:
if PYTHON_SCRIPT:

    import argparse

    parser = argparse.ArgumentParser(description="")
    parser.add_argument('--seed',default=2021,type=int)
    parser.add_argument('--fold',type=int,required=True)
    parser.add_argument('--remap',type=bool,required=True)
    parser.add_argument('--threshold',type=float,required=False)
    parser.add_argument('--mode',required=False)
    parser.add_argument('--flip',type=bool,required=False)

    # Parse the arguments
    args = parser.parse_args()

    # Run
    SEED = args.seed
    FOLD = args.fold
    RUN_REMAP = args.remap
    THRESHOLD = args.threshold
    MODE = args.mode
    FLIP = args.flip

### Folders

In [3]:
# Filepaths
OUTPUT_FOLDER = f'./fold_{FOLD}'
LOG_OUTPUT = OUTPUT_FOLDER + f'/fold-{FOLD}.log'

### Dataset Setup

In [4]:
# Dataset
TSV = r'C:\Users\ChenYi\Downloads\AAI3001_Project\labels\IEMOCAP_4.tsv'
AUDIO_DIRECTORY = r'C:\Users\ChenYi\Downloads\AAI3001_Project\small-project\IEMOCAP_full_release_audio'

### Select GPUs (For multi-GPU setup)

In [5]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### Logger

In [6]:
import logging
def get_logger(filename, verbosity=1, name=None):
    level_dict = {0: logging.DEBUG, 1: logging.INFO, 2: logging.WARNING}
    formatter = logging.Formatter("[%(asctime)s][%(filename)s][line:%(lineno)d][%(levelname)s] %(message)s")
    logger = logging.getLogger(name)
    logger.setLevel(level_dict[verbosity])

    fh = logging.FileHandler(filename, "w")
    fh.setFormatter(formatter)                                                                                                                                                                                     
    logger.addHandler(fh)                                                                                                                                                                                          
                                                                                                                                                                                                                   
    sh = logging.StreamHandler()                                                                                                                                                                                   
    sh.setFormatter(formatter)                                                                                                                                                                                     
    logger.addHandler(sh)                                                                                                                                                                                          
                                                                                                                                                                                                                   
    return logger
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
logger = get_logger(LOG_OUTPUT)

## Imports

In [7]:
import csv
import torch
import random
import librosa
import torchaudio
import contractions
import torch.nn as nn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import Dataset, DatasetDict
from torchaudio import functional as audioF
from torchaudio.transforms import Resample
from torchaudio.compliance import kaldi
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report, recall_score, accuracy_score
from transformers import WhisperProcessor, WhisperForConditionalGeneration, pipeline

[2025-01-12 09:26:01,630][config.py][line:54][INFO] PyTorch version 2.4.1+cu124 available.
[2025-01-12 09:26:01,633][config.py][line:101][INFO] TensorFlow version 2.18.0 available.


### Log Details

In [8]:
logger.info("----- Models -----")
logger.info(f"Speech-To-Text (Transcription) Model: {TRANSCRIPTION_MODEL_NAME}")
logger.info("----- Parameters -----")
logger.info(f"Seed: {SEED}")
logger.info(f"Fold: {FOLD}")
logger.info("--------------------")

[2025-01-12 09:26:06,635][1971636866.py][line:1][INFO] ----- Models -----
[2025-01-12 09:26:06,636][1971636866.py][line:2][INFO] Speech-To-Text (Transcription) Model: openai/whisper-tiny.en
[2025-01-12 09:26:06,637][1971636866.py][line:3][INFO] ----- Parameters -----
[2025-01-12 09:26:06,639][1971636866.py][line:4][INFO] Seed: 22
[2025-01-12 09:26:06,639][1971636866.py][line:5][INFO] Fold: 0
[2025-01-12 09:26:06,640][1971636866.py][line:6][INFO] --------------------


### Provided Code

In [9]:
class Pad_trunc_wav(nn.Module):
    def __init__(self, max_len: int = 6*16000):
        super(Pad_trunc_wav, self).__init__()
        self.max_len = max_len
    def forward(self,x):
        shape = x.shape
        length = shape[1]
        if length < self.max_len:
            multiple = self.max_len//length+1
            x_tmp = torch.cat((x,)*multiple, axis=1)
            x_new = x_tmp[:,0:self.max_len]
        else:
            x_new = x[:,0:self.max_len]
        return x_new

In [10]:
def setup_seed(seed=2021):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
setup_seed(SEED)

### Download Required Models

In [11]:
# Load whisper model and processor for speech-to-text
S2T_processor = WhisperProcessor.from_pretrained(TRANSCRIPTION_MODEL_NAME)
S2T_Model = WhisperForConditionalGeneration.from_pretrained(TRANSCRIPTION_MODEL_NAME)
S2T_Model = S2T_Model.to('cuda')
feature_extractor=S2T_processor.feature_extractor

logger.info(F"Speech-To-Text (Transcription) model loaded from {TRANSCRIPTION_MODEL_NAME} successfully")

[2025-01-12 09:26:09,081][440628305.py][line:7][INFO] Speech-To-Text (Transcription) model loaded from openai/whisper-tiny.en successfully


### Dataset & Loading

In [12]:
class Mydataset(Dataset):
    def __init__(self, mode='train', max_len=6, seed=2021, fold=0, data_path=TSV, audio_dir=AUDIO_DIRECTORY):
        self.mode = mode
        data_all = pd.read_csv(data_path, sep='\t')
        SpkNames = np.unique(data_all['speaker'])  # ['Ses01F', 'Ses01M', ..., 'Ses05M']
        self.data_info = self.split_dataset(data_all, fold, SpkNames, mode)
        self.get_audio_dir_path = os.path.join(audio_dir)
        self.pad_trunc = Pad_trunc_wav(max_len * 16000)
         
        # Label encoding
        self.label = self.data_info['label'].astype('category').cat.codes.values
        self.ClassNames = np.unique(self.data_info['label'])
        self.NumClasses = len(self.ClassNames)
        self.weight = 1 / torch.tensor([(self.label == i).sum() for i in range(self.NumClasses)]).float()

    def get_classname(self):
        return self.ClassNames
    
    # Updated split_dataset function using fold
    
    def split_dataset(self, df_all, fold, speakers, mode):
        
        spk_len = len(speakers)
        test_idx = np.array(df_all['speaker']==speakers[fold%spk_len])
        if fold%2==0:
            val_idx = np.array(df_all['speaker']==speakers[(fold+1)%spk_len])
        else:
            val_idx = np.array(df_all['speaker']==speakers[(fold-1)%spk_len])
        train_idx = True^(test_idx+val_idx)
        train_data_info = df_all[train_idx].reset_index(drop=True)
        val_data_info = df_all[val_idx].reset_index(drop=True)
        test_data_info = df_all[test_idx].reset_index(drop=True)

        if self.mode == 'train':
            data_info = train_data_info
        elif self.mode == 'val':
            data_info = val_data_info
        elif self.mode == 'test':
            data_info = test_data_info
        else:
            data_info = df_all
        
        logger.info(f"Mode: {mode} Fold: {fold}")
        return data_info

    def pre_process(self, wav):
        
        if self.mode == 'test': 
            return wav
        else:
            wav = self.pad_trunc(wav)
            return wav

    def __len__(self):
        return len(self.data_info)

    def __getitem__(self, idx):
        # Load the raw waveform from file using data_info to get filenames
        wav_path = os.path.join(self.get_audio_dir_path, self.data_info['filename'][idx]) + '.wav'
        wav, sample_rate = torchaudio.load(wav_path)

        # Preprocess the waveform (e.g., pad/truncate if needed)
        wav = self.pre_process(wav)

        # Apply Wav2Vec2 feature extractor
        inputs = feature_extractor(
            wav.squeeze().numpy(),  # Convert PyTorch tensor to numpy array
            sampling_rate=sample_rate,
            return_tensors="pt",  # Return PyTorch tensors
            padding=True  # Optionally pad to a fixed length
        )

        label = self.label[idx]

        # Return the processed input values and the label
        return {
            'input_values': inputs['input_values'].squeeze(0),  # Remove extra batch dimension
            'labels': torch.tensor(label, dtype=torch.long)}

In [13]:
# Instantiate datasets
test_dataset = Mydataset(mode='test', max_len=6, fold=FOLD)

logger.info("Dataset Loaded")

[2025-01-12 09:26:09,104][1700717295.py][line:43][INFO] Mode: test Fold: 0
[2025-01-12 09:26:09,105][1146238807.py][line:4][INFO] Dataset Loaded


In [14]:
# Put test information into a dataframe for later use
data_info = test_dataset.data_info
test_dataframe = data_info[['filename', 'label']].copy()
test_dataframe['filepath'] = test_dataframe['filename'].apply(
    lambda x: os.path.join(test_dataset.get_audio_dir_path, f"{x}.wav"))

## Perform Inference and Obtain Transcriptions

#### Function to clean up transcription to match what is required for compute_wer.py
- Remove full stop
- Remove all punctuations except apostrophe
- Change all to UPPERCASE

In [15]:
def format_transcription(ID, text):
    
    original_text = text  # Store the original text for comparison
    
    # Remove full stop
    text = text.replace('.', '')
    
    # Remove all punctuations except apostrophe
    text = ''.join(char if char.isalnum() or char == "'" else ' ' for char in text)
    
    # Convert all text to uppercase
    text = text.upper()

    # Alert if any formatting was done:
    if text != original_text:
        logger.info(f"Transcription of file {ID} has been formatted")
        logger.info(f"ORIGINAL: {original_text}")
        logger.info(f"FORMATTED: {text}")
    
    return text

In [16]:
def predict(test_dataframe):

    results = []
    total = test_dataframe.shape[0]
    count = 1

    # Iterate over each audio file in the test folder
    for index, row in test_dataframe.iterrows():

        # Display progress
        print(f'File {count} of {total}', end='\r')
        count += 1

        # Load audio file
        filename = row['filename'] + '.wav'
        audio_file = os.path.join(AUDIO_DIRECTORY, filename)
        audio, sample_rate = librosa.load(audio_file, sr = 16000)
        
        # Tokenize the input audio for speech-to-text model
        input_features = S2T_processor(audio, return_tensors="pt", sampling_rate=16000, padding="longest").input_features
        input_features = input_features.to('cuda')
        predicted_ids = S2T_Model.generate(input_features)
        
        transcription = S2T_processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        
        # Important!
        # Batch size should be 1 for inference. I am not using a dataloader, and iterating per file, so my batch size is effectively one
        # Padding="longest" pads each file to the longest in each batch, but because my batch is 1, effectively no padding is done.
        # For clarify, padding="do_not_pad" is used
        
        # Obtain transcriptions
        #with torch.no_grad():
        #    S2T_logits = S2T_Model(input_values).logits
        #    predicted_ids = torch.argmax(S2T_logits, dim=-1)
        #    transcription = S2T_processor.batch_decode(predicted_ids)[0]
            
        # Perform cleaning
        formatted_transcription = format_transcription(count, transcription)

        # Extract the filename without the extension
        filename = os.path.splitext(os.path.basename(audio_file))[0]

        # Append the result to the list
        results.append([filename,
                        formatted_transcription])

    logger.info(f"Done processing {total} files")

    # Write the results to a CSV file
    global CSV_FILEPATH
    CSV_FILEPATH = os.path.join(OUTPUT_FOLDER, "transcriptions.csv")

    with open(CSV_FILEPATH, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['ID', 'transcription'])
        writer.writerows(results)

    logger.info(f"Transcriptions saved to {CSV_FILEPATH}")
    return CSV_FILEPATH

In [17]:
predict(test_dataframe)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
C:\Users\ChenYi\anaconda3\envs\aai3001\Lib\site-packages\transformers\models\whisper\modeling_whisper.py:599: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
[2025-01-12 09:26:11,000][1277848060.py][line:16][INFO] Transcription of file 2 has been formatted
[2025-01-12 09:26:11,001][1277848060.py][line:17][INFO] ORIGINAL:  You're the only one who still loves his parents.
[2025-01-12 09:26:11,002][1277848060.py][line:18][INFO] FORMATTED:  YOU'RE THE ONLY ONE WHO STILL LOVES HIS PARENTS
[2025-01-12 09:26:11,104][1277848060.py][line:16][INFO] T

[2025-01-12 09:26:11,213][1277848060.py][line:16][INFO] Transcription of file 4 has been formatted
[2025-01-12 09:26:11,213][1277848060.py][line:17][INFO] ORIGINAL:  You know it's lovely here. The air is sweet.
[2025-01-12 09:26:11,214][1277848060.py][line:18][INFO] FORMATTED:  YOU KNOW IT'S LOVELY HERE THE AIR IS SWEET
[2025-01-12 09:26:11,319][1277848060.py][line:16][INFO] Transcription of file 5 has been formatted
[2025-01-12 09:26:11,320][1277848060.py][line:17][INFO] ORIGINAL:  I'm sorry, no. But I can't stay.
[2025-01-12 09:26:11,321][1277848060.py][line:18][INFO] FORMATTED:  I'M SORRY  NO BUT I CAN'T STAY
[2025-01-12 09:26:11,427][1277848060.py][line:16][INFO] Transcription of file 6 has been formatted


[2025-01-12 09:26:11,428][1277848060.py][line:17][INFO] ORIGINAL:  The one thing your mother mentioned told me to go.
[2025-01-12 09:26:11,429][1277848060.py][line:18][INFO] FORMATTED:  THE ONE THING YOUR MOTHER MENTIONED TOLD ME TO GO
[2025-01-12 09:26:11,521][1277848060.py][line:16][INFO] Transcription of file 7 has been formatted
[2025-01-12 09:26:11,522][1277848060.py][line:17][INFO] ORIGINAL:  And then you, you've been kind of...
[2025-01-12 09:26:11,523][1277848060.py][line:18][INFO] FORMATTED:  AND THEN YOU  YOU'VE BEEN KIND OF
[2025-01-12 09:26:11,612][1277848060.py][line:16][INFO] Transcription of file 8 has been formatted
[2025-01-12 09:26:11,612][1277848060.py][line:17][INFO] ORIGINAL:  I sort of embarrassed ever since I got here.


[2025-01-12 09:26:11,613][1277848060.py][line:18][INFO] FORMATTED:  I SORT OF EMBARRASSED EVER SINCE I GOT HERE
[2025-01-12 09:26:11,670][1277848060.py][line:16][INFO] Transcription of file 9 has been formatted
[2025-01-12 09:26:11,671][1277848060.py][line:17][INFO] ORIGINAL:  I knew they would.
[2025-01-12 09:26:11,672][1277848060.py][line:18][INFO] FORMATTED:  I KNEW THEY WOULD


[2025-01-12 09:26:11,810][1277848060.py][line:16][INFO] Transcription of file 10 has been formatted
[2025-01-12 09:26:11,811][1277848060.py][line:17][INFO] ORIGINAL:  For sure, from your mother's point of view. Why else would I come?
[2025-01-12 09:26:11,811][1277848060.py][line:18][INFO] FORMATTED:  FOR SURE  FROM YOUR MOTHER'S POINT OF VIEW WHY ELSE WOULD I COME 
[2025-01-12 09:26:11,900][1277848060.py][line:16][INFO] Transcription of file 11 has been formatted
[2025-01-12 09:26:11,901][1277848060.py][line:17][INFO] ORIGINAL:  I guess that's kind of why I came.
[2025-01-12 09:26:11,902][1277848060.py][line:18][INFO] FORMATTED:  I GUESS THAT'S KIND OF WHY I CAME
[2025-01-12 09:26:11,929][1277848060.py][line:16][INFO] Transcription of file 12 has been formatted
[2025-01-12 09:26:11,931][1277848060.py][line:17][INFO] ORIGINAL:  you
[2025-01-12 09:26:11,932][1277848060.py][line:18][INFO] FORMATTED:  YOU
[2025-01-12 09:26:11,980][1277848060.py][line:16][INFO] Transcription of file 13 has 

...

[2025-01-12 09:27:16,399][1277848060.py][line:16][INFO] Transcription of file 521 has been formatted
[2025-01-12 09:27:16,399][1277848060.py][line:17][INFO] ORIGINAL:  Yes, meaning just that.
[2025-01-12 09:27:16,400][1277848060.py][line:18][INFO] FORMATTED:  YES  MEANING JUST THAT
[2025-01-12 09:27:16,494][1277848060.py][line:16][INFO] Transcription of file 522 has been formatted
[2025-01-12 09:27:16,495][1277848060.py][line:17][INFO] ORIGINAL:  Don't laugh at me. I'm being serious.
[2025-01-12 09:27:16,496][1277848060.py][line:18][INFO] FORMATTED:  DON'T LAUGH AT ME I'M BEING SERIOUS
[2025-01-12 09:27:16,547][1277848060.py][line:16][INFO] Transcription of file 523 has been formatted
[2025-01-12 09:27:16,548][1277848060.py][line:17][INFO] ORIGINAL:  Who's there?
[2025-01-12 09:27:16,548][1277848060.py][line:18][INFO] FORMATTED:  WHO'S THERE 


[2025-01-12 09:27:16,627][1277848060.py][line:16][INFO] Transcription of file 524 has been formatted
[2025-01-12 09:27:16,627][1277848060.py][line:17][INFO] ORIGINAL:  I got a little bit of a good idea.
[2025-01-12 09:27:16,628][1277848060.py][line:18][INFO] FORMATTED:  I GOT A LITTLE BIT OF A GOOD IDEA
[2025-01-12 09:27:16,717][1277848060.py][line:16][INFO] Transcription of file 525 has been formatted
[2025-01-12 09:27:16,718][1277848060.py][line:17][INFO] ORIGINAL:  Oh, Elliot, where's Don't Pop?
[2025-01-12 09:27:16,719][1277848060.py][line:18][INFO] FORMATTED:  OH  ELLIOT  WHERE'S DON'T POP 


[2025-01-12 09:27:16,939][1277848060.py][line:16][INFO] Transcription of file 526 has been formatted
[2025-01-12 09:27:16,939][1277848060.py][line:17][INFO] ORIGINAL:  Thank you, darling. The same applies to you too. Except that if you so much is look at another woman, I'll kill you.
[2025-01-12 09:27:16,940][1277848060.py][line:18][INFO] FORMATTED:  THANK YOU  DARLING THE SAME APPLIES TO YOU TOO EXCEPT THAT IF YOU SO MUCH IS LOOK AT ANOTHER WOMAN  I'LL KILL YOU
[2025-01-12 09:27:17,083][1277848060.py][line:16][INFO] Transcription of file 527 has been formatted
[2025-01-12 09:27:17,084][1277848060.py][line:17][INFO] ORIGINAL:  Oh, Charles. Oh, that was his name. We did wiggle so beautifully.
[2025-01-12 09:27:17,085][1277848060.py][line:18][INFO] FORMATTED:  OH  CHARLES OH  THAT WAS HIS NAME WE DID WIGGLE SO BEAUTIFULLY


[2025-01-12 09:27:17,327][1277848060.py][line:16][INFO] Transcription of file 528 has been formatted
[2025-01-12 09:27:17,328][1277848060.py][line:17][INFO] ORIGINAL:  The manager came in, he found this rolling about the floor, biting and scratching like panthers. Oh, no, no. So, honestly, utterly ridiculous.
[2025-01-12 09:27:17,329][1277848060.py][line:18][INFO] FORMATTED:  THE MANAGER CAME IN  HE FOUND THIS ROLLING ABOUT THE FLOOR  BITING AND SCRATCHING LIKE PANTHERS OH  NO  NO SO  HONESTLY  UTTERLY RIDICULOUS
[2025-01-12 09:27:17,463][1277848060.py][line:16][INFO] Transcription of file 529 has been formatted
[2025-01-12 09:27:17,464][1277848060.py][line:17][INFO] ORIGINAL:  We want to have a chunk of that. Very much, still here.
[2025-01-12 09:27:17,465][1277848060.py][line:18][INFO] FORMATTED:  WE WANT TO HAVE A CHUNK OF THAT VERY MUCH  STILL HERE
[2025-01-12 09:27:17,466][3904740078.py][line:47][INFO] Done processing 528 files
[2025-01-12 09:27:17,468][3904740078.py][line:58][INF

'./fold_0\\transcriptions.csv'

## Convert CSV into TXT file
- Optional step to convert into TXT file for use in compute_wer.py
- This function can also be found in notebooks/TranscriptionFormatter.ipynb, where you can optionaly convert each fold at once

In [18]:
# Input and output file paths
input_csv = CSV_FILEPATH  # Replace with your CSV file path
output_txt = OUTPUT_FOLDER + "/transcriptions.txt"  # Replace with your desired TXT file path

In [19]:
# Open the CSV file and read its content
def convert_csv_to_txt(input_csv, output_txt):
    with open(input_csv, mode="r", encoding="utf-8") as csv_file:
        csv_reader = csv.reader(csv_file)
        header = next(csv_reader, None)  # Skip header if present
        with open(output_txt, mode="w", encoding="utf-8") as txt_file:
            for row in csv_reader:
                # Combine ID and Transcript with a space separator
                txt_file.write(f"{row[0]} {row[1]}\n")
    
convert_csv_to_txt(input_csv, output_txt)
print(f"Converted and formatted {input_csv} to {output_txt} for use in computing word error rate.")

Converted and formatted ./fold_0\transcriptions.csv to ./fold_0/transcriptions.txt for use in computing word error rate.
